In [1]:
import cv2
import numpy as np

# hsv range for markers
hsvcolors = [[119,87,192,179,193,255],
             [85,118,173,159,177,255],
             [0,65,182,47,153,255]]

# color for markers
colorvalues = [[0,255,0],   # bgr
               [255,0,0],
               [0,0,255]]

In [2]:
# return the coordinates and rgb color of marker detected

def findcolor(img, hsvcolors, colorvalues):
    imghsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    count = 0
    newpoints =[]
    # check for every marker's color
    for color in hsvcolors:
        lower = np.array(color[0:3])
        upper = np.array(color[3:6])
        mask = cv2.inRange(imghsv, lower, upper)
        x,y = getContours(mask)
        if x != 0 and y != 0:
            newpoints.append([x,y,count])
        count += 1
    return newpoints

In [3]:
# find coordinates of marker

def getContours(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x,y,w,h = 0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 200:
            peri = cv2.arcLength(cnt, True)
            epsilon = 0.1*peri
            approx = cv2.approxPolyDP(cnt,epsilon,True)

            x,y,w,h = cv2.boundingRect(approx)
    return x+w//2,y+h//2 # middle point

In [4]:
# draw on background

def drawoncanvas(points, colorvalues):
    for point in points:
        cv2.circle(imgresult, (point[0],point[1]) , 5, colorvalues[point[2]], cv2.FILLED)

In [5]:
cap = cv2.VideoCapture(0)

cap.set(3,640) # width
cap.set(4,480) # height
cap.set(10,150) # brightness

# to keep history of points which was drawn earlier 
# because image visible to us keep on changing for every frame 

pastpoints = [] # [x,y,colorindex]

while True:
    success, img = cap.read() # success: image read successfully or not
    
    # white background
    imgresult = np.empty((img.shape))
    imgresult.fill(255)
    
    newpoints = findcolor(img, hsvcolors, colorvalues)
    
    if len(newpoints)!=0:
        for newp in newpoints:
            pastpoints.append(newp)
            
    if len(pastpoints)!=0:
        drawoncanvas(pastpoints, colorvalues)
        
    cv2.imshow("Video",imgresult)
    
    if (cv2.waitKey(5) & 0xFF) == ord('q'):  # close on presssing q
        cv2.destroyAllWindows()
        break;
        
cap.release()